In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Exploring business file

In [ ]:
df_b = pd.read_json('yelp_data/business.json', lines=True)
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)


In [ ]:
# 1 = open, 0 = closed
df_b = df_b[df_b['is_open']==1]
#df_b = df_b[df_b['state']=='AZ']

In [ ]:
#exploring cities
df_b.city.value_counts().sort_values(ascending=False).head(5)

In [ ]:
df_b =(df_b.loc[df_b['city'].isin(['Las Vegas', 'Phoenix','Charlotte','Scottsdale'  ])])

In [ ]:
df_b.shape

In [ ]:
#dropping columns
drop_columns = ['hours','is_open']
df_b = df_b.drop(drop_columns, axis=1)

In [ ]:
#selecting resturants
business_resturants = df_b[df_b['categories'].str.contains(
              'Restaurants',
              case=False, na=False)]


In [ ]:
#business_resturants.head(2)
business_resturants.name.nunique()

In [ ]:
# for i in str(col):
#     print(business_resturants['categories'][i])

In [ ]:
business_resturants.categories.value_counts().sort_values(ascending=False).head(50)

In [ ]:
# using regex to remove Restaurants
import re
#removing Resturants, 
business_resturants['categories'] = [re.sub(r'Restaurants\,\s', "", x) for x in business_resturants['categories']]
#removing , Resturants
business_resturants['categories'] = [re.sub(r',\ \bRestaurants', "", x) for x in business_resturants['categories']]
 #changing American (Traditional) and American (New) to American
business_resturants['categories'] = [re.sub(r'American\s\(\w*\)', "American", x) for x in business_resturants['categories']]


In [ ]:
# categories = ['Pizza', 
#               'Italian', 
#               'Tex-Mex', 
#               'Seafood', 
#               'Mexican', 
#               'Thai', 
#               'Breakfast & Brunch', 
#               'Burgers', 
#               'Bars', 
#               'Juice Bars & Smoothies', 
#               'Coffee & Tea', 
#               'American (Traditional)',
#               'American (New)',
#               'Sandwiches',
#               'Salad',
#               'Chinese']

# newFeatures2 = pd.DataFrame(0, index=np.arange(len(resDataFeatures)), columns=categories)

# for i in range(0,len(resDataFeatures)):
#     for x, j in enumerate(categories):
#         if j in resDataFeatures['categories'][i]:
#             newFeatures2.set_value(i, j, 1)

# newFeatures2.head(3)

In [ ]:
df_explode = business_resturants.assign(categories = business_resturants.categories
                         .str.split(', ')).explode('categories')
df_explode.categories.value_counts().sort_values(ascending=False).head(20)

In [ ]:
df_explode[df_explode.categories=='Food']

In [ ]:
business_resturants.categories.value_counts().sort_values(ascending=False).head(50)

In [ ]:
#business_resturants['categories'] =business_resturants['categories'].str.replace(',','')
#creating new column by extracting Fast Food from string of words
business_resturants['new_categories']= business_resturants['categories'].str.extract(r'(Fast Food)')
# Bringing rest of the original categories to fill nan in new categor column
business_resturants.new_categories.fillna(business_resturants.categories, inplace=True)


In [ ]:
business_resturants.new_categories.value_counts().sort_values(ascending=False).head(50)
#changing order
business_resturants['new_categories'] = [re.sub("(Pizza, Italian)", "Italian, Pizza", x) for x in business_resturants['new_categories']]
business_resturants['new_categories'] = [re.sub("(Food, Mexican)|(Mexican, Food)", "Mexican", x) for x in business_resturants['new_categories']]
business_resturants['new_categories'] = [re.sub("(Seafood, Mexican)", "Mexican, Seafood", x) for x in business_resturants['new_categories']]
business_resturants.new_categories.value_counts().sort_values(ascending=False).head(50)

In [ ]:
business_resturants.new_categories.value_counts()

# Exploring Attributes

In [ ]:
import json  
from pandas.io.json import json_normalize 
import numpy as np

In [ ]:
business_resturants =business_resturants[business_resturants['attributes'].notnull()]

In [ ]:
#getting price range from nested attribute column
idx = business_resturants.index[business_resturants.attributes.notna()]
price_range = pd.json_normalize(business_resturants.attributes.dropna().tolist())['RestaurantsPriceRange2']
price_range.index =idx
business_resturants['price_range'] =price_range

In [ ]:
#getting ambience from nested Json column attibutes
idx = business_resturants.index[business_resturants.attributes.notna()]
ambience = pd.json_normalize(business_resturants.attributes.dropna().tolist())['Ambience']
ambience.index =idx
business_resturants['ambience'] = ambience

In [ ]:
# idx = business_resturants.index[business_resturants.attributes.notna()]
# romantic = pd.json_normalize(business_resturants.attributes.dropna().tolist(), max_level = 1) #max_level to look at nested level
# romantic.index =idx
# business_resturants['romantic'] = romantic

In [ ]:
# type(business_resturants.loc[11, 'attributes'])

In [ ]:
import ast 

In [ ]:
business_resturants.head().ambience.dropna().apply(lambda x: ast.literal_eval(x)['romantic'])

In [ ]:
def get_ambience(x,ambience):
    try:
        return(ast.literal_eval(x)[ambience])
    except:
        pass
    
        

In [ ]:
business_resturants['romantic'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'romantic'))
business_resturants['intimate'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'intimate'))
business_resturants['touristy'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'touristy'))
business_resturants['hipster'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'hipster'))
business_resturants['divey'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'divey'))
business_resturants['classy'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'classy'))
business_resturants['trendy'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'trendy'))
business_resturants['upscale'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'upscale'))
business_resturants['casual'] = business_resturants.ambience.apply(lambda x: get_ambience(x,'casual'))


In [ ]:
#changing True or False to 1&0
business_resturants.replace(False,0, inplace =True)
business_resturants.replace(True,1, inplace =True)
#df[['cl1','cl2']]=(fault_diagnostics[['cl1','cl2']]=='True').astype(int)
#test.replace(False, 0, inplace=True)

In [ ]:
#dropping columns
business_resturants = business_resturants.drop(['attributes','ambience', 'categories' ], axis =1)

In [ ]:
# checking key for dictionary
# def getList(dict): 
#     return dict.keys() 

In [ ]:
business_resturants =business_resturants.replace(to_replace ='None', value =np.nan)

In [ ]:
business_resturants['price_range'] =  pd.to_numeric(business_resturants["price_range"])

In [ ]:
impute_list =['price_range', 'romantic', 'intimate' , 'touristy' , 'hipster', 'divey', 'classy', 
              'trendy', 'upscale', 'casual'   ]
# Impute missing
business_resturants = business_resturants.fillna(business_resturants.groupby('name')[impute_list].transform('median'))
business_resturants=business_resturants.dropna(subset =impute_list)

In [ ]:
def binary_cat(x):
    if x in ('Fast Food'):
        category = 1
    else:
        category = 0
    return category

business_resturants['category'] = business_resturants['new_categories'].apply (binary_cat)

In [ ]:
#Creating business_resturants
business_resturants.to_csv('yelp_data/business_resturants.csv', index =False)

In [ ]:
#ENTITY TyPE

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md') 


In [ ]:
doc = nlp(str(business_resturants['new_categories']))
print([(ent.text, ent.label_) for ent in doc.ents])

In [ ]:
spacy.explain('NORP')

In [ ]:
#Extarcting entity type from the dataframe

def extract_category(text):
    for ent in nlp(text).ents:
        if ent.label_ == 'NORP':
            return(ent.text)
#         if ent.label_ =='NORP':
#             return(ent.text)
        
        else:
            return (text)
        
            
                
                

In [ ]:
def add_column(df):
    business_resturants['ethnic_categories'] = business_resturants['new_categories'].apply(extract_category)

In [ ]:
add_column(business_resturants)

In [ ]:
missing_df =business_resturants[business_resturants.new_categories.isna()]

In [ ]:
missing_df.head()

In [ ]:
business_resturants['new_categories'].value_counts(dropna =False).sort_values(ascending=False).head(30)
# .plot(kind ='barh')
# plt.title('Ethnic Resturant Value Count');

In [ ]:
business_resturants.name.value_counts(dropna=False).sort_values(ascending=False).head()

In [ ]:
business_resturants[business_resturants.name=='Taco Bell']

In [ ]:
business_resturants[business_resturants.new_categories=="Event Planning & Services"]

In [ ]:
#getting the restaurants  less then 2
new =business_resturants.name.value_counts().reset_index(name="count").query("count < 2")

In [ ]:
new.head()

In [ ]:
#Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
sns.countplot(business_resturants['stars'], ax=ax)
plt.title('Star Ratings');

In [ ]:
g = sns.catplot(x="stars", col="city",
                data=business_resturants, kind="count",
                height= 5, aspect=.9);


In [ ]:
# df_explode.categories.nunique()

In [ ]:
# # find category with resturant
# df_explode[df_explode.categories.str.contains('Restaurants',
#                       case=True,na=False)].categories.value_counts()

In [ ]:
# df_review.head()

# Merging business file with review and user info

In [ ]:
business_resturants = pd.read_csv('yelp_data/business_resturants.csv')

In [ ]:
review_json_path  = 'yelp_data/review.json'


In [ ]:
review_json_path  = 'yelp_data/review.json'
size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)


In [ ]:
# Merging business file and review file
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(business_resturants, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
business_review = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [ ]:
business_review.to_csv('yelp_data/yelp_merged.csv', index =False)

In [ ]:
yelp_merged =pd.read_csv('yelp_data/yelp_merged.csv')

In [ ]:
yelp_merged.head()

In [ ]:
yelp_merged.state.value_counts()

In [ ]:
yelp_merged[yelp_merged.state =="NM"]

In [ ]:
user_json_path = 'yelp_data/user.json'
size = 1000000
user = pd.read_json(user_json_path, lines=True,
                      dtype={'user_id':str,
                             'name':str,'review_count':int,'yelping_since':str,'friends':str,'useful':int,
                             'funny':int,'cool':int,'fans':int, 'elite':int,'average_stars':float,'compliment_hot':int,
                             'compliment_more':int, 'compliment_profile':int,'compliment_cute':int,'compliment_list':int,
                             'compliment_note':int, 'compliment_plain':int, 'compliment_cool':int,'compliment_funny':int,
                             'compliment_funny':int, 'compliment_writer':int,'compliment_photos': int
                            },
                      chunksize=size)

In [ ]:
# Merging user data
chunk_list = []
for chunk_user in user:
    # Drop columns that aren't needed
    chunk_user = chunk_user.drop(['name','friends','useful', 'funny','cool','fans', 'elite','compliment_hot',
                             'compliment_more', 'compliment_profile','compliment_cute','compliment_list',
                             'compliment_note', 'compliment_plain', 'compliment_cool','compliment_funny',
                             'compliment_funny', 'compliment_writer','compliment_photos', 'yelping_since'], axis=1)
    # Renaming column name to avoid conflict with business/review overall star rating
    chunk_user = chunk_user.rename(columns={'average_stars': 'user__average_stars'})
    # Inner merge with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(yelp_merged, chunk_user, on='user_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related users")
    chunk_list.append(chunk_merged)
#creating a folder with chunks of merged file

# After trimming down the review file, concatenate all relevant data back to one dataframe
user_merged = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [ ]:
# #creating a folder in pc with chunks of merged file rather than appending the list
# chunk_list = []
# i=1
# for chunk_user in user:
#     # Drop columns that aren't needed
#     chunk_user = chunk_user.drop(['name','friends','useful', 'funny','cool','fans', 'elite','compliment_hot',
#                              'compliment_more', 'compliment_profile','compliment_cute','compliment_list',
#                              'compliment_note', 'compliment_plain', 'compliment_cool','compliment_funny',
#                              'compliment_funny', 'compliment_writer','compliment_photos'], axis=1)
#     # Renaming column name to avoid conflict with business/review overall star rating
#     chunk_user = chunk_user.rename(columns={'average_stars': 'user__average_stars'})
#     # Inner merge with  business with reviewers file so only users related to the review remains
#     chunk_merged = pd.merge(yelp_merged, chunk_user, on='user_id', how='inner')
#     # Show feedback on progress
#     print(f"{chunk_merged.shape[0]} out of {size:,} related users")
#     chunk_merged.to_csv('yelp_data/chunk/chunk_{}.csv'.format(i), index = False)
# # # After trimming down the review file, concatenate all relevant data back to one dataframe
# # user_merged = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [ ]:
# keeping users with more than 1 reviewers 1413908-1371908= 42000
user_mered= user_merged[user_merged.review_count>1]
user_merged.info()

In [ ]:
user_merged.to_csv('yelp_data/user_merged.csv', index =False)

In [ ]:
user_merged =pd.read_csv('yelp_data/user_merged.csv')

In [ ]:
user_merged.info()